In [ ]:
import avro
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

In [ ]:
import gdal
import numpy
zonefilename = "examples/data/states.shp"
datafilename = "examples/data/glc2000.tif"

In [ ]:
schema = avro.schema.Parse(open("geotiff.avsc", "rb").read())
writer = DataFileWriter(open("geotiff.avro", "wb"), DatumWriter(), schema)

In [ ]:
filehandle = gdal.Open(datafilename)
if filehandle is None:
    print ('Unable to open %s', datafilename)
    sys.exit(1)
Y = filehandle.RasterYSize
X = filehandle.RasterXSize
band = filehandle.GetRasterBand(1)
no_data_value = band.GetNoDataValue()
data_array = band.ReadAsArray(0,0,X,Y)

In [ ]:
writer.append({"data": data_array, "nodatavalue": no_data_value, "x":X, "y":Y})
writer.close()

In [ ]:
import pyspark
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("Test")
sc = pyspark.SparkContext(conf = conf)

In [ ]:
df = pyspark.read.format("com.databricks.spark.avro").load("geotiff.avro")